In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)

In [4]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
tool

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\harshit.vashistha\\LLMs\\LangChain\\my_env\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [6]:
tool.name

'wikipedia'

In [9]:
from langchain_community.vectorstores import FAISS
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

In [10]:
loader = WebBaseLoader('https://docs.smith.langchain.com/')

In [11]:
documents = loader.load()

In [12]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': "LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain's open source frameworks is not necessary - LangSmith works on its own!", 'language': 'en'}, page_content='\n\n\n\n\nGet started with LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Cloud)AdministrationSelf-hostingReferenceQuick StartOn this pageGet started with LangSmith\nLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain\'s open source frameworks is not necessary - LangSmith works on it

In [15]:
chunks = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)

In [16]:
document_chunk = chunks.split_documents(documents)

In [17]:
document_chunk

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': "LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain's open source frameworks is not necessary - LangSmith works on its own!", 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': "LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain's open source frameworks is not necessary - LangSmith works on its own!", 'language': 'en'}, page_content="Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick 

In [18]:
embeddings = OllamaEmbeddings(
    model = 'llama3.2:1b'
)

In [19]:
db = FAISS.from_documents(document_chunk, embedding=embeddings)

In [23]:
retriever = db.as_retriever()

In [21]:
from langchain_ollama.llms import OllamaLLM

In [22]:
llm = OllamaLLM(
    model = 'llama3.2:1b'
)

In [24]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000012E86FFA930>, search_kwargs={})

In [25]:
from langchain.tools.retriever import create_retriever_tool

In [26]:
retriever_tool = create_retriever_tool(retriever, "langsmith_search", "Search for information about Langsmith, For any question related to langsmith you must use this tool")

In [27]:
retriever_tool.name

'langsmith_search'

In [28]:
tool.name

'wikipedia'

In [29]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [30]:
api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)

In [31]:
arxiv_tool = ArxivQueryRun(api_wrapper=api_wrapper)

In [32]:
arxiv_tool.name

'arxiv'

In [33]:
tools = [retriever_tool, tool, arxiv_tool]

In [34]:
tools

[Tool(name='langsmith_search', description='Search for information about Langsmith, For any question related to langsmith you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000012EF9C92CA0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000012E86FFA930>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000012EF9CB4360>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000012E86FFA930>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, temp

In [45]:
from langchain_core.prompts import ChatPromptTemplate

In [55]:
prompt = ChatPromptTemplate.from_template(
    """
    Answer the questions based on the tools I have provided to you.
    {agent_scratchpad}
    Question: {input}
    """
)

In [63]:
from langchain.agents import create_openai_tools_agent

In [64]:
agents = create_openai_tools_agent(llm, tools, prompt)

In [51]:
from langchain.agents import AgentExecutor

In [66]:
agent_executor = AgentExecutor(agent=agents, tools=tools, verbose=True, handle_parsing_errors=True)

In [67]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template='\n    Answer the questions based on the tools I have provided to you.\n    {agent_scratchpad}\n    Question: {input}\n    '), additional_kwargs={})])
| RunnableBinding(bound=OllamaLLM(model='llama3.2:1b'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'langsmith_search', 'description': 'Search for information about Langsmith, For any question related to langsmith you must use this tool', 'parameters': {'properties': {'query': {'description': 'query to look up in retriever', 'type': 'string'}}, 'required': ['query'], 'type':

In [68]:
response = agent_executor.invoke({'input': 'What is langsmith'})



> Entering new AgentExecutor chain...


ValueError: This output parser only works on ChatGeneration output